In [1]:
import cfr
job = cfr.ReconJob()

# proxy database: fetching & preprocessing
job.load_proxydb('PAGES2kv2')
job.filter_proxydb(by='ptype', keys=['coral'])
job.annualize_proxydb(months=[12, 1, 2], ptypes=['coral'])

# model prior: fetching & preprocessing
job.load_clim(tag='prior', path_dict={'tas': 'iCESM_past1000historical/tas'}, anom_period=(1951, 1980))
job.load_clim(tag='obs', path_dict={'tas': 'storical/tas'}, anom_period=(1951, 1980))

# proxy system modeling
job.calib_psms(
    ptype_psm_dict={'coral.d18O': 'Linear', 'coral.calc': 'Linear', 'coral.SrCa': 'Linear'},
    ptype_season_dict={'coral.d18O': [12, 1, 2], 'coral.calc': [12, 1, 2], 'coral.SrCa': [12, 1, 2]},
    calib_period=(1850, 2015),
)
job.forward_psms()

# model prior: processing
job.annualize_clim(tag='prior', months=[12, 1, 2])
job.regrid_clim(tag='prior', nlat=42, nlon=63)
job.crop_clim(tag='prior', lat_min=-35, lat_max=35)

# paleo data assimilation
job.run_da_mc(save_dirpath='./recons/lmr-real-pages2k', recon_seeds=list(range(1, 11)))

# reconstruction validation
res = cfr.ReconRes('./recons/lmr-real-pages2k')
res.load(['nino3.4', 'tas'])

target = cfr.ClimateField().fetch('20CRv3/tas', vn='air').rename('tas').get_anom((1951, 1980))
target = target.annualize(months=[12, 1, 2]).crop(lat_min=-35, lat_max=35)

# validate the prior field against 20CR
stat = 'corr'
valid_fd = job.prior['tas'].compare(target, stat=stat, timespan=(1874, 2000))
fig, ax = valid_fd.plot(
    title=f'{stat}(prior, obs), mean={valid_fd.geo_mean().value[0,0]:.2f}',
    projection='PlateCarree', latlon_range=(-32, 32, 0, 360), plot_cbar=False,
)

# validate the reconstructed field against 20CR
valid_fd = res.recons['tas'].compare(target, stat=stat, timespan=(1874, 2000))
valid_fd.plot_kwargs.update({'cbar_orientation': 'horizontal', 'cbar_pad': 0.1})
fig, ax = valid_fd.plot(
    title=f'{stat}(recon, obs), mean={valid_fd.geo_mean().value[0,0]:.2f}',
    projection='PlateCarree', latlon_range=(-32, 32, 0, 360), plot_cbar=True,
    plot_proxydb=True, proxydb=job.proxydb.filter(by='tag', keys=['calibrated']),
    proxydb_lgd_kws={'loc': 'lower left', 'bbox_to_anchor': (1, 0)},
)

# validate the reconstructed NINO3.4 against Bunge & Clark (2009)
bc09 = cfr.EnsTS().fetch('BC09_NINO34').annualize(months=[12, 1, 2])
fig, ax = res.recons['nino3.4'].compare(bc09, timespan=(1874, 2000)).plot_qs()
ax.set_xlim(1600, 2000)
ax.set_ylabel('NINO3.4 [K]')